# 金融异常检测任务

## 1. 实验介绍

反欺诈是金融行业永恒的主题，在互联网金融信贷业务中，数字金融反欺诈技术已经得到广泛应用并取得良好效果，这其中包括了近几年迅速发展并在各个领域
得到越来越广泛应用的神经网络。本项目以互联网智能风控为背景，从用户相互关联和影响的视角，探索满足风控反欺诈领域需求的，可拓展、高效的神经
网络应用方案，从而帮助更好地识别欺诈用户。

本项目主要关于实现预测模型(**项目用图神经网络举例，具体实现可以使用其他模型**)，进行节点异常检测任务，并验证模型精度。而本项目基于的数据集[DGraph](https://dgraph.xinye.com/introduction)，[DGraph](https://dgraph.xinye.com/introduction)
是大规模动态图数据集的集合，由真实金融场景中随着时间演变事件和标签构成。

### 1.1 实验目的

- 了解如何使用Pytorch进行神经网络训练
- 了解如何使用Pytorch-geometric等图网络深度学习库进行简单图神经网络设计(推荐使用GAT, GraphSAGE模型)。
- 了解如何利用MO平台进行模型性能评估。

### 1.2 预备知识
- 具备一定的深度学习理论知识，如卷积神经网络、损失函数、优化器，训练策略等。
- 了解并熟悉Pytorch计算框架。
- 学习Pytorch-geometric，请前往：https://pytorch-geometric.readthedocs.io/en/latest/
    
### 1.3实验环境
- numpy = 1.26.4  
- pytorch = 2.3.1  
- torch_geometric = 2.5.3  
- torch_scatter = 2.1.2  
- torch_sparse = 0.6.18  

## 2. 实验内容

### 2.1 数据集信息
DGraph-Fin 是一个由数百万个节点和边组成的有向无边权的动态图。它代表了Finvolution Group用户之间的社交网络，其中一个节点对应一个Finvolution 用户，从一个用户到另一个用户的边表示**该用户将另一个用户视为紧急联系人**。
下面是`位于dataset/DGraphFin目录`的DGraphFin数据集的描述:
```
x:  20维节点特征向量
y:  节点对应标签，一共包含四类。其中类1代表欺诈用户而类0代表正常用户(实验中需要进行预测的两类标签)，类2和类3则是背景用户，即无需预测其标签。
edge_index:  图数据边集,每条边的形式(id_a,id_b)，其中ids是x中的索引
edge_type: 共11种类型的边
edge_timestamp: 脱敏后的时间戳
train_mask, valid_mask, test_mask: 训练集，验证集和测试集掩码
```
本预测任务为识别欺诈用户的节点预测任务,只需要将欺诈用户（Class 1）从正常用户（Class 0）中区分出来。需要注意的是，其中测试集中样本对应的label**均被标记为-100**。

### 2.2 导入相关包

导入相应模块，设置数据集路径、设备等。

In [1]:
from utils import DGraphFin
from utils.utils import prepare_folder
from utils.evaluator import Evaluator

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T

import numpy as np
from torch_geometric.data import Data
import os

#设置gpu设备
device = 0
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)


### 2.3 数据处理

在使用数据集训练网络前，首先需要对数据进行归一化等预处理，如下：

In [2]:
path='./datasets/632d74d4e2843a53167ee9a1-momodel/' #数据保存路径
save_dir='./results/' #模型保存路径
dataset_name='DGraph'
dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor())

nlabels = dataset.num_classes
if dataset_name in ['DGraph']:
    nlabels = 2    #本实验中仅需预测类0和类1

data = dataset[0]
data.adj_t = data.adj_t.to_symmetric() #将有向图转化为无向图


if dataset_name in ['DGraph']:
    x = data.x
    x = (x - x.mean(0)) / x.std(0)
    data.x = x
if data.y.dim() == 2:
    data.y = data.y.squeeze(1)

split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  #划分训练集，验证集
print(split_idx['train'].shape)
print(split_idx['valid'].shape)
print(split_idx['test'].shape)

train_idx = split_idx['train']
result_dir = prepare_folder(dataset_name,'mlp')
# convert data.adj_t(adjacent matrix type=SparseTensor) to edge_index shape=(2, |E|) 
# as input for SAGEConv operator
edge_index = torch.stack([data.adj_t.storage.row(), data.adj_t.storage.col()], dim=0)
data["edge_index"] = edge_index
print(edge_index.shape)
data = Data(x=data.x, edge_index=data.edge_index, y=data.y)
print(data)
print(nlabels)

torch.Size([857899])
torch.Size([183862])
torch.Size([183840])
torch.Size([2, 7994520])
Data(x=[3700550, 20], edge_index=[2, 7994520], y=[3700550])
2


In [3]:
from torch_geometric.loader import NeighborLoader

# process train/test/valid split of nodes and edges
train_loader = NeighborLoader(
    data,
    num_neighbors=[2],
    input_nodes=split_idx['train'],
    batch_size=len(split_idx['train']), # TODO: smaller batch_size
    shuffle=True
)
test_loader = NeighborLoader(
    data,
    num_neighbors=[2],
    input_nodes=split_idx['test'],
    batch_size=len(split_idx['test']), # TODO
    shuffle=True
)
valid_loader = NeighborLoader(
    data,
    num_neighbors=[2],
    input_nodes=split_idx['valid'],
    batch_size=len(split_idx['valid']), # TODO
    shuffle=True
)
for batch in train_loader:
    print(batch)
    break
for batch in test_loader:
    print(batch)
    break
for batch in valid_loader:
    print(batch)
    break

/home/zbw/miniconda3/envs/zju-ai-course/lib/python3.9/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


Data(x=[1751951, 20], edge_index=[2, 1481532], y=[1751951], n_id=[1751951], e_id=[1481532], input_id=[857899], batch_size=857899)
Data(x=[469344, 20], edge_index=[2, 317603], y=[469344], n_id=[469344], e_id=[317603], input_id=[183840], batch_size=183840)
Data(x=[469481, 20], edge_index=[2, 317762], y=[469481], n_id=[469481], e_id=[317762], input_id=[183862], batch_size=183862)


这里我们可以查看数据各部分维度

In [4]:
print(data)
print(data.x.shape)  #feature
print(data.y.shape)  #label

Data(x=[3700550, 20], edge_index=[2, 7994520], y=[3700550])
torch.Size([3700550, 20])
torch.Size([3700550])


### 2.4 定义模型

使用GNN模型，同时考虑节点和信息及其边信息作为模型输入

In [5]:
from torch_geometric.nn import SAGEConv

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(GraphSAGE, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        # 第一层
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        # 中间层
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        # 输出层
        self.convs.append(SAGEConv(hidden_channels, out_channels))
        
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return F.log_softmax(x, dim=-1)

配置后续训练、验证、推理用到的参数。可以调整以下超参以提高模型训练后的验证精度：

- `epochs`：在训练集上训练的代数；
- `lr`：学习率；
- `num_layers`：网络的层数；
- `hidden_channels`：隐藏层维数；
- `dropout`：dropout比例；
- `weight_decay`：正则化项的系数。

In [6]:
# 模型参数配置
sage_parameters = {
    'lr': 0.01,
    'num_layers': 3,
    'hidden_channels': 64,
    'dropout': 0.5,
    'weight_decay': 1e-5
}
epochs = 500
log_steps =10 # log记录周期


In [7]:
# 初始化模型
para_dict = sage_parameters
model_para = sage_parameters.copy()
model_para.pop('lr')
model_para.pop('weight_decay')
model = GraphSAGE(
    in_channels=data.x.size(-1),
    hidden_channels=sage_parameters['hidden_channels'],
    out_channels=nlabels,
    num_layers=sage_parameters['num_layers'],
    dropout=sage_parameters['dropout']
).to(device)
print(f'Model GraphSAGE initialized')

eval_metric = 'auc'  #使用AUC衡量指标
evaluator = Evaluator(eval_metric)

Model GraphSAGE initialized


初始化模型，并使用**Area Under the Curve (AUC)** 作为模型评价指标来衡量模型的表现。AUC通过对ROC曲线下各部分的面积求和而得。

具体计算过程参见 https://github.com/scikit-learn/scikit-learn/blob/baf828ca1/sklearn/metrics/_ranking.py#L363

### 2.5 训练

使用训练集中的节点用于训练模型，并使用验证集进行挑选模型。

In [8]:
def train(model, data, train_idx, optimizer):
     # data.y is labels of shape (N, )
    model.train()

    loss_list = torch.tensor([], device=device)
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()

        out = model(batch.x, batch.edge_index)
    # out = model(data.x[train_idx])
        # 仅预测0和1的标签，去除 label 为 2,3,-100 的样本
        mask = batch.y != 2
        mask = mask & (batch.y != 3)
        mask = mask & (batch.y != -100)
        loss = F.nll_loss(out[mask], batch.y[mask])
        
        loss.backward()
        loss_list = torch.cat([loss_list, loss.unsqueeze(0).detach()])
        optimizer.step()

    return loss_list.mean()
    # return loss.item()


In [9]:
def test(model, data, split_idx, evaluator):
    # data.y is labels of shape (N, )
    with torch.no_grad():
        model.eval()

        losses, eval_results = dict(), dict()
        for key in ['train', 'valid']:
            if key == 'train':
                loader = train_loader
            else:
                loader = valid_loader
        #     node_id = split_idx[key]            
            for batch in loader:
                out = model(batch.x, batch.edge_index)
                # out = model(data.x[node_id])
                y_pred = out.exp()  # (N,num_classes)

                # 仅预测0和1的标签，去除 label 为 2,3,-100 的样本
                mask = batch.y != 2
                mask = mask & (batch.y != 3)
                mask = mask & (batch.y != -100)
                losses[key] = F.nll_loss(out[mask], batch.y[mask]).item()
                eval_results[key] = evaluator.eval(batch.y[mask], y_pred[mask])[eval_metric]

    return eval_results, losses, y_pred


In [10]:
print(sum(p.numel() for p in model.parameters()))  #模型总参数量

model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=para_dict['lr'], weight_decay=para_dict['weight_decay'])
best_valid = 0
min_valid_loss = 1e8

data = data.to(device)

for epoch in range(1,epochs + 1):
    loss = train(model, data, train_idx, optimizer)
    eval_results, losses, out = test(model, data, split_idx, evaluator)
    train_eval, valid_eval = eval_results['train'], eval_results['valid']
    train_loss, valid_loss = losses['train'], losses['valid']

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), save_dir+'/model.pt') #将表现最好的模型保存

    if epoch % log_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_eval:.3f}, ' # 我们将AUC值乘上100，使其在0-100的区间内
              f'Valid: {100 * valid_eval:.3f} ')


11138
Epoch: 10, Loss: 0.1302, Train: 62.813, Valid: 54.756 
Epoch: 20, Loss: 0.0861, Train: 66.104, Valid: 58.757 
Epoch: 30, Loss: 0.0751, Train: 68.884, Valid: 61.992 
Epoch: 40, Loss: 0.0717, Train: 70.497, Valid: 65.556 
Epoch: 50, Loss: 0.0695, Train: 71.229, Valid: 66.209 
Epoch: 60, Loss: 0.0680, Train: 72.556, Valid: 68.847 
Epoch: 70, Loss: 0.0673, Train: 73.004, Valid: 69.595 
Epoch: 80, Loss: 0.0665, Train: 73.529, Valid: 70.738 
Epoch: 90, Loss: 0.0661, Train: 73.991, Valid: 71.744 
Epoch: 100, Loss: 0.0656, Train: 74.396, Valid: 72.539 
Epoch: 110, Loss: 0.0651, Train: 74.669, Valid: 73.300 
Epoch: 120, Loss: 0.0651, Train: 74.907, Valid: 73.523 
Epoch: 130, Loss: 0.0649, Train: 75.158, Valid: 73.698 
Epoch: 140, Loss: 0.0647, Train: 75.165, Valid: 73.871 
Epoch: 150, Loss: 0.0646, Train: 75.281, Valid: 74.087 
Epoch: 160, Loss: 0.0644, Train: 75.374, Valid: 74.113 
Epoch: 170, Loss: 0.0643, Train: 75.533, Valid: 74.366 
Epoch: 180, Loss: 0.0641, Train: 75.641, Valid: 74.

### 2.6 模型预测

In [11]:
model.load_state_dict(torch.load(save_dir+'/model.pt')) #载入验证集上表现最好的模型
node_id_result_mapping = model(data.x, data.edge_index) # 所有节点全部预测一遍，然后直接查表
print(node_id_result_mapping.shape)

def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        # out = model(data.x[node_id])
        out = node_id_result_mapping[node_id]
        y_pred = out.exp()  # (N,num_classes)

    return y_pred


torch.Size([3700550, 2])


In [15]:
dic={0:"正常用户",1:"欺诈用户"}
node_idx = 0
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

node_idx = 1
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')


tensor([0.9841, 0.0159])
节点 0 预测对应的标签为:0, 为正常用户。
tensor([0.9862, 0.0138])
节点 1 预测对应的标签为:0, 为正常用户。


保存所有节点的预测结果，快速答案评测

In [13]:
import pickle
model.load_state_dict(torch.load(save_dir+'/model.pt')) #载入验证集上表现最好的模型
node_id_result_mapping = model(data.x, data.edge_index).to('cpu')
pickle.dump(node_id_result_mapping, open('node_id_result_mapping.pkl', 'wb'))

## 3. 作业评分

**作业要求**：    
                         
1. 请加载你认为训练最佳的模型（不限于图神经网络)
2. 提交的作业包括【程序报告.pdf】和代码文件。

**注意：**
          
1. 在训练模型等过程中如果需要**保存数据、模型**等请写到 **results** 文件夹，如果采用 [离线任务](https://momodel.cn/docs/#/zh-cn/%E5%9C%A8GPU%E6%88%96CPU%E8%B5%84%E6%BA%90%E4%B8%8A%E8%AE%AD%E7%BB%83%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E6%A8%A1%E5%9E%8B) 请务必将模型保存在 **results** 文件夹下。
2. 训练出自己最好的模型后，先按照下列 cell 操作方式实现 NoteBook 加载模型测试；请测试通过在进行【系统测试】。
3. 点击左侧栏`提交作业`后点击`生成文件`则只需勾选 `predict()` 函数的cell，即【**模型预测代码答题区域**】的 cell。
4. 请导入必要的包和第三方库 (包括此文件中曾经导入过的)。
5. 请加载你认为训练最佳的模型，即请按要求填写**模型路径**。
6. `predict()`函数的输入和输出请不要改动。

===========================================  **模型预测代码答题区域**  =========================================== 

在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [2]:
## 生成 main.py 时请勾选此 cell
from utils import DGraphFin
from utils.evaluator import Evaluator
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.data import Data
import numpy as np
import os
import pickle

# model.load_state_dict(torch.load(save_dir+'/model.pt')) #载入验证集上表现最好的模型
# node_id_result_mapping = model(data.x)

# lookup table
node_id_result_mapping = pickle.load(open('node_id_result_mapping.pkl', 'rb'))

def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        # out = model(data.x[node_id])
        out = node_id_result_mapping[node_id]
        y_pred = out.exp()  # (N,num_classes)

    return y_pred
